In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import precision_score, recall_score, f1_score
import time

# Load a dummy dataset (MNIST)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Configure TensorFlow to allocate GPU memory on an as-needed basis
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth for each GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

# Training loop with GPU VRAM tracking for the RNN model
rnn_model = models.Sequential([
    layers.SimpleRNN(128, activation='relu', input_shape=(28, 28)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

total_vram_usage = 0
total_f1 = 0
num_checks = 5

# Train the RNN model
for epoch in range(num_checks):
    start_time = time.time()

    with tf.device('/GPU:0'):
        rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        rnn_model.fit(x_train, y_train, epochs=1, verbose=2)

        # Predict on the test set
        y_pred = rnn_model.predict(x_test)
        y_pred_classes = tf.argmax(y_pred, axis=1).numpy()

        # Calculate precision, recall, and F1 score
        precision = precision_score(y_test, y_pred_classes, average='weighted')
        recall = recall_score(y_test, y_pred_classes, average='weighted')
        f1 = f1_score(y_test, y_pred_classes, average='weighted')

        # Print information
        print(f"RNN Model, Epoch {epoch + 1}/5 - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    end_time = time.time()

    # Check GPU usage and extract VRAM usage
    gpu_info = !nvidia-smi
    vram_usage_line = next((line for line in gpu_info if 'MiB /' in line), None)

    if vram_usage_line:
        # Extract numerical parts and convert to integers
        vram_used_str = vram_usage_line.split()[8]
        vram_used = int(vram_used_str[:-3])  # Remove 'MiB' and convert to int

        # Print VRAM usage information
        print(f"VRAM Usage: {vram_used} MiB")

        total_vram_usage += vram_used
        total_f1 += f1

        # Optional: Add a delay to avoid overwhelming the GPU monitoring tool
        time.sleep(5)

# Calculate and print average VRAM usage
average_vram_usage = total_vram_usage / num_checks
average_f1 = total_f1 / num_checks
print(f"Average VRAM Usage: {average_vram_usage} MiB")
print(f"Average F1 Score: {round(average_f1, 2)}")



11490434/11490434 [==============================] - 1s 0us/step
1 Physical GPUs, 1 Logical GPUs
1875/1875 - 16s - loss: 0.4314 - accuracy: 0.8634 - 16s/epoch - 9ms/step
313/313 [==============================] - 2s 4ms/step
RNN Model, Epoch 1/5 - Precision: 0.9454, Recall: 0.9442, F1 Score: 0.9443
VRAM Usage: 639 MiB
1875/1875 - 16s - loss: 0.1993 - accuracy: 0.9422 - 16s/epoch - 8ms/step
313/313 [==============================] - 2s 4ms/step
RNN Model, Epoch 2/5 - Precision: 0.9562, Recall: 0.9552, F1 Score: 0.9552
VRAM Usage: 1151 MiB
1875/1875 - 17s - loss: 0.1666 - accuracy: 0.9531 - 17s/epoch - 9ms/step
313/313 [==============================] - 1s 4ms/step
RNN Model, Epoch 3/5 - Precision: 0.9522, Recall: 0.9506, F1 Score: 0.9508
VRAM Usage: 1151 MiB
1875/1875 - 15s - loss: 0.1486 - accuracy: 0.9573 - 15s/epoch - 8ms/step
313/313 [==============================] - 2s 7ms/step
RNN Model, Epoch 4/5 - Precision: 0.9657, Recall: 0.9654, F1 Score: 0.9654
VRAM Usage: 1151 MiB
1875/187